# question 2

In [190]:
from os import path
import pandas as pd
import numpy as np
import gurobipy as gb
from gurobipy import GRB
from gurobipy import quicksum
import math

In [174]:
%cd /Users/eason/Desktop/OperationReseach/Case2
work_dir = '/Users/eason/Desktop/OperationReseach/Case2'
data_path = path.join(work_dir, 'data')
data_file = path.join(data_path, 's1.xlsx')

/Users/eason/Desktop/OperationReseach/Case2


In [175]:
Demand_df = pd.read_excel(data_file, sheet_name='Demand', index_col='Product')
Init_df = pd.read_excel(data_file, sheet_name='Initial inventory', index_col='Product')
ShippingCost_df = pd.read_excel(data_file, sheet_name='Shipping cost', index_col='Product')
InTransit_df = pd.read_excel(data_file, sheet_name='In-transit', index_col='Product')
CBM_df = pd.read_excel(data_file, sheet_name='Size', index_col='Product')
PriceAndCost_df = pd.read_excel(data_file, sheet_name='Price and cost', index_col='Product')
Shortage_df = pd.read_excel(data_file, sheet_name='Shortage', index_col='Product')
VendorProduct_df = pd.read_excel(data_file, sheet_name='Vendor-Product', index_col='Product')
VendorCost_df = pd.read_excel(data_file, sheet_name='Vendor cost', index_col='Vendor')
Bound_df = pd.read_excel(data_file, sheet_name='Bounds', index_col='Product')
Conflict_df = pd.read_excel(data_file, sheet_name='Conflict', index_col='Conflict')

N, M = Demand_df.shape
K = 3 #(express delivery, air freight, Ocean freight)
V = VendorCost_df.shape[0]
ContainerCap = 30 #Case 1 data
ContainerCost = 2750 #Case 1 data

Demand_ij = Demand_df.to_numpy()
Init_i = Init_df.to_numpy().squeeze()
BuyCost_i = PriceAndCost_df['Purchasing cost'].to_numpy()
HoldCost_i = PriceAndCost_df['Holding cost'].to_numpy()
Transit_ij = InTransit_df.to_numpy()
ShipFixedCost_k = np.array([100, 80, 50]) #Case 1 data
ShipVarCost_ik = np.concatenate(( ShippingCost_df.to_numpy(), np.zeros((N,1)) ), axis=1)
CBM_i = CBM_df.to_numpy().squeeze()
LostSaleCost_i = Shortage_df['Lost sales'].to_numpy()
BackOrderCost_i = Shortage_df['Backorder'].to_numpy()
BackOrderProb_i = Shortage_df['Backorder percentage'].to_numpy()
VendorFixedCost_v = VendorCost_df.to_numpy().squeeze()
MinOrder_i = Bound_df.to_numpy().squeeze()
ConflictPair_alpha = Conflict_df.to_numpy()
ProductVendor_i = VendorProduct_df.to_numpy().squeeze()
M_big = np.sum(Demand_ij) + sum(Init_i)

In [176]:
#model
instance_name = 's1'
model = gb.Model(name=instance_name)

In [177]:
#decision variable
x = model.addVars(N, M, K, vtype=GRB.INTEGER,name='x')
Abin = model.addVars(M, K, vtype=GRB.BINARY, name='Abin')
ContainerCnt = model.addVars(M, vtype=GRB.INTEGER, name='ContainerCnt')
StockLevel = model.addVars(N, M, vtype=GRB.CONTINUOUS, name='StockLevel')
UnfulDemand = model.addVars(N, M, vtype=GRB.CONTINUOUS, name='UnfulDemand')
Bbin = model.addVars(N, M, vtype=GRB.BINARY, name='Bbin')
Cbin = model.addVars(N, M, vtype=GRB.BINARY, name='Cbin')
Dbin = model.addVars(V, M, vtype=GRB.BINARY, name='Dbin')

In [178]:
#Expr
VolumeInOceanExpr_j = [
gb.LinExpr(
    quicksum(x[i,j,2] * CBM_i[i] for i in range(N))
)    
for j in range(M)]

BackOrderCntExpr_ij = [
    [
        gb.LinExpr( UnfulDemand[i,j] * BackOrderProb_i[i] )
    for j in range(M)]
for i in range(N)]

LostSaleCntExpr_ij = [
    [
        gb.LinExpr( UnfulDemand[i,j] * (1 - BackOrderProb_i[i]) )
    for j in range(M)]
for i in range(N)]

EndingInventoryExpr_ij = [
    [
        gb.LinExpr( Init_i[i] - Demand_ij[i,0] )
    ] +
    [
        gb.LinExpr(
            StockLevel[i,0] + x[i,0,0] + Transit_ij[i,1] - Demand_ij[i,1] - BackOrderCntExpr_ij[i][0]
        )
    ] +
    [
        gb.LinExpr(
            StockLevel[i,1] + x[i,1,0] + x[i,0,1] + Transit_ij[i,2] - Demand_ij[i,2] - BackOrderCntExpr_ij[i][1]
        )
    ] +
    [
        gb.LinExpr(
            StockLevel[i,j-1] + x[i,j-1,0] + x[i,j-2,1] + x[i,j-3,2] - Demand_ij[i,j] - BackOrderCntExpr_ij[i][j-1]
        )
    for j in range(3, M)]
for i in range(N)]

In [179]:
#obj function
TotalPurchaseCost = gb.LinExpr(
    quicksum(
        quicksum(
            quicksum(
                x[i,j,k] * BuyCost_i[i]
            for i in range(N))
        for j in range(M))
    for k in range(K))
)

TotalShipFixedCost = gb.LinExpr(
    quicksum(
        quicksum(
            Abin[j,k] * ShipFixedCost_k[k]
        for k in range(K))
    for j in range(M))
)

TotalShipVarCost = gb.LinExpr(
    quicksum(
        quicksum(
            quicksum(
                x[i,j,k] * ShipVarCost_ik[i,k]
            for k in range(K))
        for j in range(M))
    for i in range(N))
)

TotalHoldingCost = gb.LinExpr(
    quicksum(
        quicksum(
            StockLevel[i,j] * HoldCost_i[i]
        for i in range(N))
    for j in range(M))
)

TotalContainerCost = gb.LinExpr(
    quicksum(ContainerCnt[j] for j in range(M)) * ContainerCost
)

TotalBackOrderCost = gb.LinExpr(
    quicksum(
        quicksum(
            BackOrderCntExpr_ij[i][j] * BackOrderCost_i[i]
        for i in range(N))
    for j in range(M))
)

TotalLostSaleCost = gb.LinExpr(
    quicksum(
        quicksum(
            LostSaleCntExpr_ij[i][j] * LostSaleCost_i[i]
        for i in range(N))
    for j in range(M))
)

TotalVendorFixedCost = gb.LinExpr(
    quicksum(
        quicksum(
            Dbin[v,j] * VendorFixedCost_v[v]
        for v in range(V))
    for j in range(M))
)

In [180]:
#set obj function
model.setObjective(
    TotalPurchaseCost +
    TotalShipFixedCost + 
    TotalShipVarCost + 
    TotalHoldingCost + 
    TotalContainerCost + 
    TotalBackOrderCost + 
    TotalLostSaleCost +
    TotalVendorFixedCost
)

In [181]:
#Contrain

#let Abin to behave correctly
_ = model.addConstrs(
    quicksum(x[i,j,k] for i in range(N)) / M_big <= Abin[j,k]
for j in range(M)
for k in range(K))

#let ContainerCnt to behave correctly
_ = model.addConstrs(
    VolumeInOceanExpr_j[j] / ContainerCap <= ContainerCnt[j]
for j in range(M))

#let stock level & UnfulDemand behave correctly
_ = model.addConstrs(
    EndingInventoryExpr_ij[i][j] <= StockLevel[i,j]
for i in range(N)
for j in range(M))
_ = model.addConstrs(
    -EndingInventoryExpr_ij[i][j] <= UnfulDemand[i,j]
for i in range(N)
for j in range(M))

_ = model.addConstrs(
    StockLevel[i,j] <= 0 + M_big * Bbin[i,j]
for i in range(N)
for j in range(M))
_ = model.addConstrs(
    StockLevel[i,j] <= EndingInventoryExpr_ij[i][j] + M_big * (1-Bbin[i,j])
for i in range(N)
for j in range(M))
_ = model.addConstrs(
    UnfulDemand[i,j] <= 0 + M_big * (1-Bbin[i,j])
for i in range(N)
for j in range(M))
_ = model.addConstrs(
    UnfulDemand[i,j] <= -EndingInventoryExpr_ij[i][j] + M_big * Bbin[i,j]
for i in range(N)
for j in range(M))

#let Cbin behave correctly
_ = model.addConstrs(
    quicksum(x[i,j,k] for k in range(K)) / M_big <= Cbin[i,j]
for i in range(N)
for j in range(M))

#let Dbin behave correctly
_ = model.addConstrs(
    quicksum(
        quicksum(x[i,j,k] for k in range(K))
    for i in range(N) if ProductVendor_i[i] == v) / M_big <= Dbin[v,j]
for v in range(V)
for j in range(M))

#Minumin order Bound
_ = model.addConstrs(
    Cbin[i,j] * MinOrder_i[i] <= quicksum(x[i,j,k] for k in range(K))
for i in range(N)
for j in range(M))

#conflict
_ = model.addConstrs(
    Cbin[a,j] + Cbin[b,j] <= 1
for j in range(M)
for a, b in ConflictPair_alpha)

In [182]:
model.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 21190 rows, 18356 columns and 109790 nonzeros
Model fingerprint: 0xd70d815a
Variable types: 5200 continuous, 13156 integer (5330 binary)
Coefficient statistics:
  Matrix range     [3e-06, 4e+05]
  Objective range  [1e+01, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+05]
Presolve removed 5185 rows and 3057 columns
Presolve time: 0.36s
Presolved: 16005 rows, 15299 columns, 90017 nonzeros
Variable types: 4269 continuous, 11030 integer (3407 binary)
Found heuristic solution: objective 3.712039e+09

Root relaxation: objective 1.269782e+08, 4747 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.2698e+08    0 2160 3.7120e+09 1.2698e+08  96.6%     -    1s
H    0     0           

In [215]:
print('Express delivery')
df = pd.DataFrame([[int(x[i,j,0].x) for j in range(M)] for i in range(N)], columns=range(M))
display(df)
print('Air frieght')
df = pd.DataFrame([[int(x[i,j,1].x) for j in range(M)] for i in range(N)], columns=range(M))
display(df)
print('Ocean frieght')
df = pd.DataFrame([[int(x[i,j,2].x) for j in range(M)] for i in range(N)], columns=range(M))
display(df)

Express delivery


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,133,0,0,0,0,...,0,0,0,0,8,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,0,0,0,270,0,136,0,129,...,0,0,0,0,0,0,0,0,283,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,0,0,0,0,0,0,0,0,0,0,...,377,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,172,0,0,0


Air frieght


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0,0,0,0,0,0,81,159,0,168,...,0,0,0,0,108,0,56,0,0,0
1,0,0,0,0,0,0,203,185,0,0,...,0,0,496,0,0,0,0,0,0,0
2,0,0,0,200,0,0,0,0,436,0,...,0,0,0,0,174,0,0,0,0,0
3,0,0,0,0,0,0,0,630,253,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,193,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,13,0,84,0,...,0,0,28,2,0,333,0,0,0,0
97,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,170,0,0,0,0
98,0,0,0,0,0,86,0,0,0,0,...,5,0,0,0,0,4,0,0,0,0


Ocean frieght


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0,0,38,91,0,70,6,225,12,0,...,67,0,148,10,166,10,0,0,0,0
1,0,0,0,127,0,0,0,0,0,0,...,0,0,0,0,227,0,0,0,0,0
2,0,82,0,102,50,0,0,0,12,0,...,0,0,0,0,332,0,187,0,0,0
3,0,97,107,0,0,0,0,26,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,67,0,0,0,0,0,612,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,113,0,0,77,0,21,0,0,...,0,410,0,156,136,255,0,0,0,0
96,0,0,0,0,190,42,5,11,216,0,...,185,23,9,0,32,2,236,0,0,0
97,0,0,0,0,0,0,0,0,0,0,...,322,0,0,147,8,237,0,0,0,0
98,0,52,143,30,21,0,0,0,0,0,...,38,17,0,0,8,0,20,0,0,0


In [217]:
print('Ending Inventory')
df = pd.DataFrame([[EndingInventoryExpr_ij[i][j].getValue() for j in range(M)] for i in range(N)], columns=range(M))
display(df)

Ending Inventory


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,662.0,607.0,435.0,241.0,147.0,0.00,12.00,-2.273737e-13,0.000,0.0000,...,1.430000e+02,0.000000e+00,0.000000,6.700000e+01,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,410.0,357.0,289.0,104.0,91.0,-45.00,-160.50,-2.573500e+02,-0.145,-0.1015,...,1.983154e+02,-6.845514e-01,-180.479186,-2.823354e+02,242.365199,-0.634801,-4.443608e-01,-3.110525e-01,-1.542177e+02,-2.699524e+02
2,346.0,167.0,166.0,117.0,0.0,0.00,169.00,2.000000e+01,-81.000,-8.1000,...,2.669997e+02,5.999692e+00,5.999692,-1.900003e+02,-80.000031,-0.000003,-3.080995e-07,-3.080995e-08,-3.080995e-09,-3.080995e-10
3,208.0,258.0,180.0,49.0,0.0,-48.00,-201.00,-2.760000e+02,-384.000,107.0000,...,-1.395506e-11,-2.490000e+02,-290.000000,-2.900000e+02,-477.000000,-706.000000,-8.030000e+02,-1.029000e+03,-1.089000e+03,-1.089000e+03
4,365.0,303.0,220.0,130.0,0.0,-49.00,-120.00,-1.910000e+02,-353.000,-409.0000,...,0.000000e+00,0.000000e+00,-1.000000,3.233254e-10,0.000000,-13.000000,-3.500000e+02,-3.630000e+02,-3.630000e+02,-3.980000e+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,481.0,322.0,256.0,84.0,0.0,0.00,156.00,0.000000e+00,0.000,-163.0000,...,-3.320078e+02,-2.660039e+02,-330.001953,-4.160010e+02,-0.000488,-108.000244,-1.220703e-04,-6.103516e-05,-3.051758e-05,-1.525879e-05
96,761.0,582.0,560.0,508.0,396.0,101.00,9.00,0.000000e+00,0.000,5.0000,...,-1.800000e-03,1.359995e+02,-0.000540,4.099984e+01,39.999838,-50.000162,-1.600000e+02,-1.458000e-05,-6.400000e+01,-2.000013e-01
97,709.0,638.0,737.0,683.0,570.0,461.00,449.00,1.940000e+02,194.000,170.0000,...,-2.689840e+02,-1.872000e-01,-0.149760,2.988019e+01,-0.119808,-0.095846,-7.667712e-02,-6.134170e-02,2.369509e+02,-2.104907e+01
98,213.0,253.0,184.0,0.0,0.0,0.00,-32.00,-3.800000e+01,-86.000,-180.0000,...,1.310000e+02,2.842171e-13,0.000000,0.000000e+00,-46.000000,-138.000000,-2.200000e+02,0.000000e+00,0.000000e+00,1.000000e+01


In [226]:
print(f'TotalPurchaseCost    :{TotalPurchaseCost.getValue():.2f}')
print(f'TotalShipFixedCost   :{TotalShipFixedCost.getValue():.2f}')
print(f'TotalShipVarCost     :{TotalShipVarCost.getValue():.2f}')
print(f'TotalHoldingCost     :{TotalHoldingCost.getValue():.2f}')
print(f'TotalContainerCost   :{TotalContainerCost.getValue():.2f}')
print(f'TotalBackOrderCost   :{TotalBackOrderCost.getValue():.2f}')
print(f'TotalLostSaleCost    :{TotalLostSaleCost.getValue():.2f}')
print(f'TotalVendorFixedCost :{TotalVendorFixedCost.getValue():.2f}')
print(f'objective-value      :{model.ObjVal:.2f}')

TotalPurchaseCost    :1038115400.00
TotalShipFixedCost   :5570.00
TotalShipVarCost     :5909359.00
TotalHoldingCost     :21123054.84
TotalContainerCost   :918500.00
TotalBackOrderCost   :84484046.44
TotalLostSaleCost    :602133084.25
TotalVendorFixedCost :19200.00
objective-value      :1752708214.53
